In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import os
import glob
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

2025-05-12 17:10:42.944775: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 17:10:42.947784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 17:10:42.955803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747044642.968633   22819 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747044642.972773   22819 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747044642.983686   22819 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
data = pd.read_csv(r'/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGMT/do_an/train_info.csv')

In [5]:
def filter_data(data, path):
    filtered_data = data[data['style'].isin(['Impressionism', 'Realism', 'Romanticism'])].copy()
    df = filtered_data[['filename', 'style']]

    # Bỏ đuôi nếu đã có .jpg, rồi thêm đúng đuôi lại
    df['filename_with_ext'] = df['filename'].apply(lambda x: os.path.splitext(x)[0] + '.jpg')
    df['full_path'] = df['filename_with_ext'].apply(lambda x: os.path.join(path, x))

    # Lọc bỏ các ảnh không tồn tại thật trên ổ đĩa
    df = df[df['full_path'].apply(os.path.exists)]

    return df


In [6]:
df = filter_data(data, '/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGMT/do_an/train')
df

/tmp/ipykernel_22819/3313668290.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['filename_with_ext'] = df['filename'].apply(lambda x: os.path.splitext(x)[0] + '.jpg')
/tmp/ipykernel_22819/3313668290.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_path'] = df['filename_with_ext'].apply(lambda x: os.path.join(path, x))


,filename,style,filename_with_ext,full_path
16,28908.jpg,Realism,28908.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
19,35766.jpg,Romanticism,35766.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
77,19834.jpg,Realism,19834.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
80,45276.jpg,Realism,45276.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
101,23209.jpg,Realism,23209.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
...,...,...,...,...
79225,35120.jpg,Romanticism,35120.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
79271,15380.jpg,Impressionism,15380.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
79296,14830.jpg,Romanticism,14830.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...
79351,2854.jpg,Impressionism,2854.jpg,/media/minhman/01DA5FE5725E67B0/Code_HK6/NMTGM...


In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.preprocessing import LabelEncoder

# 1. Chuẩn bị dữ liệu
image_paths = df['full_path'].tolist()
labels = df['style'].tolist()

# Encode nhãn nếu là text
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# 2. Dataset TensorFlow
path_ds = tf.data.Dataset.from_tensor_slices((image_paths, encoded_labels))

def load_and_preprocess_image(path, label, target_size=(224, 224)):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = preprocess_input(image)
    return image, label

image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(32)

# 3. Tải mô hình DenseNet201 (không có phần classification cuối)
densenet_model = DenseNet201(weights='imagenet', include_top=False, pooling='avg')

# 4. Trích xuất đặc trưng
all_features = []
all_labels = []

for batch_images, batch_labels in image_ds:
    features = densenet_model(batch_images, training=False)  # (batch_size, 1920)
    all_features.append(features.numpy())
    all_labels.append(batch_labels.numpy())

X = np.concatenate(all_features, axis=0)
y = np.concatenate(all_labels, axis=0)

# print("Feature shape:", X.shape)  # (num_samples, 1920)

# # 5. Train classifier với LightGBM
# clf = lgb.LGBMClassifier()
# clf.fit(X, y)

# # 6. Dự đoán
# y_pred = clf.predict(X)


2025-05-12 17:11:06.626778: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-05-12 17:14:27.227568: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: known incorrect sRGB profile
2025-05-12 17:16:06.586521: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 283306404 exceeds 10% of free system memory.
2025-05-12 17:24:27.944902: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: known incorrect sRGB profile
2025-05-12 17:31:07.771260: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
# Lưu tên ảnh vào list
image_names = df['filename'].tolist()

# Gộp tất cả đặc trưng và nhãn từ các batch
features_array = np.vstack(all_features)  # (num_samples, 1920)
labels_array = np.concatenate(all_labels)  # (num_samples,)

# Tạo DataFrame từ đặc trưng, nhãn và tên ảnh
df_features = pd.DataFrame(features_array)
df_features['label'] = labels_array
df_features['filename'] = image_names  # Thêm tên ảnh vào DataFrame

# Lưu vào file CSV
df_features.to_csv('densenet_features_with_filenames.csv', index=False)